# Exercise Set 7

This exercise set covers Pandas basics.

## Exercise 1

In [105]:
from urllib.request import urlopen
import pandas as pd
import math
import string
import statistics

gages = ['06803495','06803486']
start_date = '2000-10-1'
end_date = '2019-9-30'
starts = start_date.split('-')
ends = end_date.split('-')
start = int(starts[0])
end = int(ends[0])
years = range(start,end+1,1)

url = 'https://waterdata.usgs.gov/nwis/dv?&cb_00060=on&format=rdb&'

for i in range(len(gages)):
    str1 = 'site_no={}&'.format(gages[i])
    url = url + str1

period = 'period=&begin_date={}&end_date={}'.format(start_date,end_date)
url = url + period

data = pd.read_csv(url,sep = '\t',names = ['Agency','Site_No','DateTime','Flow','Value'],comment = '#')
data = data.drop(data[data['DateTime'] == '20d'].index)
data = data.drop(data[data['DateTime'] == 'datetime'].index)
data.to_csv('Site_Runoff_Data.csv',sep = ',')
data = data.drop('Agency',axis = 1)
data = data.drop('Value', axis = 1)
# data = data.drop('Site_No',axis = 1)
print(data[0:10])

data['DateTime'] = pd.to_datetime(data['DateTime'])
time_index_data = data.set_index('DateTime')
print(time_index_data[0:10])

time_index_data['Flow'] = time_index_data['Flow'].apply(pd.to_numeric,errors = 'coerce')
# time_index_data['Site_No'] = time_index_data['Site_No'].apply(pd.to_numeric,errors = 'coerce')
annual_means_bysite = time_index_data.groupby('Site_No').resample('A').mean()
print(annual_means_bysite.round(2))

monthly_mean_bysite = time_index_data.groupby(by = ['Site_No',time_index_data.index.month]).mean()
monthly_mean_bysite.rename(columns={'Flow':'Mean'},inplace=True)
monthly_max_bysite = time_index_data.groupby(by = ['Site_No',time_index_data.index.month]).max()
monthly_max_bysite.rename(columns={'Flow':'Max'},inplace=True)
monthly_min_bysite = time_index_data.groupby(by = ['Site_No',time_index_data.index.month]).min()
monthly_min_bysite.rename(columns={'Flow':'Min'},inplace=True)
monthly_stats_bysite = pd.concat([monthly_mean_bysite,monthly_max_bysite,monthly_min_bysite],axis = 1)
monthly_stats_bysite.index.names = ['Site_No','Month']
monthly_stats_bysite = monthly_stats_bysite.round(2)
monthly_stats_bysite.to_csv('Monthly_Runoff_Data.csv',sep = ',')
print(monthly_stats_bysite)

Site_No    DateTime  Flow
2   06803486  2000-10-01  3.30
3   06803486  2000-10-02  3.80
4   06803486  2000-10-03  4.10
5   06803486  2000-10-04  3.50
6   06803486  2000-10-05  6.30
7   06803486  2000-10-06  6.10
8   06803486  2000-10-07  7.00
9   06803486  2000-10-08  8.00
10  06803486  2000-10-09  9.00
11  06803486  2000-10-10  10.0
             Site_No  Flow
DateTime                  
2000-10-01  06803486  3.30
2000-10-02  06803486  3.80
2000-10-03  06803486  4.10
2000-10-04  06803486  3.50
2000-10-05  06803486  6.30
2000-10-06  06803486  6.10
2000-10-07  06803486  7.00
2000-10-08  06803486  8.00
2000-10-09  06803486  9.00
2000-10-10  06803486  10.0
                       Flow
Site_No  DateTime          
06803486 2000-12-31   12.06
         2001-12-31   47.57
         2002-12-31   18.64
         2003-12-31   15.00
         2004-12-31   12.25
         2005-12-31   15.78
         2006-12-31   17.37
         2007-12-31   86.75
         2008-12-31  132.08
         2009-12-31   42.96
    

## Exercise 2

In [190]:
import pandas as pd
import numpy as np

user_list = pd.read_csv('C:/Users/jstansell2/Box/M.S/Coursework/Summer 2020/NRES 898/python2020summer/jstansell/users.zip', sep = '|', compression = 'zip')
user_list.set_index('user_id',inplace = True)

occupation_counts = user_list.groupby('occupation')['occupation'].count()
stem_occupations = ['doctor','engineer','healthcare','programmer','scientist','technician']
idx = occupation_counts.index
stem_inds = idx.isin(stem_occupations)
stem_total = occupation_counts.where(stem_inds).sum()
nonstem_inds = ~idx.isin(stem_occupations)
nonstem_total = occupation_counts.where(nonstem_inds).sum()
print(f'There are {stem_total:.0f} people in STEM professions and {nonstem_total:.0f} people in non-STEM professions.' )

programmers_under35 = user_list[(user_list['occupation'] == 'programmer') & (user_list['age'] <= 35)]
male_prog_under35 = programmers_under35[programmers_under35['gender'] == 'M']
under35_males = len(male_prog_under35)
female_prog_under35 = programmers_under35[programmers_under35['gender'] == 'F']
under35_females = len(female_prog_under35)
ratio_MF_under35 = under35_males/under35_females
programmers_over35 = user_list[(user_list['occupation'] == 'programmer') & (user_list['age'] >= 35)]
male_prog_over35 = programmers_over35[programmers_over35['gender'] == 'M']
over35_males = len(male_prog_over35)
female_prog_over35 = programmers_over35[programmers_over35['gender'] == 'F']
over35_females = len(female_prog_over35)
ratio_MF_over35 = over35_males/over35_females

if ratio_MF_over35 > ratio_MF_under35:
    print(f'The ratio of male to female programmers over 35 is greater than that for the under 35 age group, at {ratio_MF_over35} and {ratio_MF_under35}, respectively.')
elif ratio_MF_over35 < ratio_MF_under35:
    print(f'The ratio of male to female programmers over 35 is less than that for the under 35 age group, at {ratio_MF_over35} and {ratio_MF_under35}, respectively.')
else:
    print(f'The ratio of male to female programmers is equal for both the over 35 and under 35 age groups at {ratio_MF_over35}.')

occupation_MF_counts = user_list.groupby(['occupation','gender']).count()['age']
occupation_MF_counts.rename('Male Female Counts')
print(occupation_MF_counts)

occupation_ages = user_list.groupby('occupation').mean()['age']
print(occupation_ages)
occupation_max_age = occupation_ages.idxmax()
occupation_min_age = occupation_ages.idxmin()
print(f'The occupation with the oldest mean age is {occupation_max_age} and the occupation with the youngest mean age is {occupation_min_age}.')

user_list['zip_initial'] = user_list['zip_code'].str[:2]
zipcode_counts = user_list.groupby('zip_initial').count()['zip_code']
zipcode_counts.rename('Area Counts')
max_user_region = zipcode_counts.idxmax()
print(f'The region with the largest number of users has zipcodes beginning with the first two digits {max_user_region}.')

There are 214 people in STEM professions and 729 people in non-STEM professions.
The ratio of male to female programmers over 35 is less than that for the under 35 age group, at 7.0 and 10.25, respectively.
occupation     gender
administrator  F          36
               M          43
artist         F          13
               M          15
doctor         M           7
educator       F          26
               M          69
engineer       F           2
               M          65
entertainment  F           2
               M          16
executive      F           3
               M          29
healthcare     F          11
               M           5
homemaker      F           6
               M           1
lawyer         F           2
               M          10
librarian      F          29
               M          22
marketing      F          10
               M          16
none           F           4
               M           5
other          F          36
               M 